In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.4.11 imported


## Generate database

In [2]:
f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179))

array([[  0,   0,   0],
       [  0,   0,   7],
       [  0,   0,  14],
       ...,
       [ 84,  84, 161],
       [ 84,  84, 168],
       [ 84,  84, 175]])

In [3]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))
database.euler.shape

(4394, 3)

In [4]:
dataset = pd.DataFrame(
    {
        "Euler1": database.euler[:, 0],
        "Euler2": database.euler[:, 1],
        "Euler3": database.euler[:, 2],
    }
)
dataset

,Euler1,Euler2,Euler3
0,0,0,0
1,0,0,7
2,0,0,14
3,0,0,21
4,0,0,28
...,...,...,...
4389,84,84,147
4390,84,84,154
4391,84,84,161
4392,84,84,168


In [5]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [6]:
# create a random generator
rgn = np.random.default_rng()

# set standard deviations for T and azimuths
std_T = 0.03
std_azimuths = 0.5

**2024-04-13 test**:  
std_T = 0.03  
std_azimuths = 0.5  

**2024-04-18 test**:  
std_T = 0.03  
std_azimuths = 0.25

**2024-04-19 test2**:  
std_T = 0.015  
std_azimuths = 0.5

**2024-04-30 test3**:  
std_T = 0.03  
std_azimuths = 0.5  
Note: more datapoints

In [7]:
# just a test
np.arange(0, 180, 6.5)

array([  0. ,   6.5,  13. ,  19.5,  26. ,  32.5,  39. ,  45.5,  52. ,
        58.5,  65. ,  71.5,  78. ,  84.5,  91. ,  97.5, 104. , 110.5,
       117. , 123.5, 130. , 136.5, 143. , 149.5, 156. , 162.5, 169. ,
       175.5])

In [8]:
round(180/6.5), round(180/5.7)

(28, 32)

In [9]:
steps = (6.5, 5.7)  # in degrees
columns = ["n28", "n32"]

for i, step in enumerate(steps):
    # create database
    angles = np.arange(0, 180, step)
    T_vals = []
    azi_vals = []

    for euler in database.euler:
        # rotate
        x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

        # extract XY intersection
        xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

        # get the indexes of specific angles
        indexes = f.find_nearest(xy_vectors["angles"], angles)

        # append values
        T_vals.append(xy_vectors.loc[indexes, ["T"]].T.values.tolist()[0])
        azi_vals.append(xy_vectors.loc[indexes, ["angles"]].T.values.tolist()[0])

    database.T_values = np.array(T_vals)
    database.azimuths = np.array(azi_vals)

    # add Gaussian noise to data
    database.T_noise = database.T_values + rgn.normal(0, std_T, database.T_values.shape)
    database.azi_noise = database.azimuths + rgn.normal(
        0, std_azimuths, database.azimuths.shape
    )

    # initialize variables
    no_noise = np.empty(database.euler.shape[0])
    noise = np.empty(database.euler.shape[0])

    for index, orientation in enumerate(database.euler):
        measures1 = np.column_stack(
            (
                database.T_values[index],
                database.azimuths[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate1 = f.find_orientation(measurements=measures1, params=(Ta, Tb, Tc))
        no_noise[index] = f.calc_disorientation(orientation, estimate1.x)

        measures2 = np.column_stack(
            (
                database.T_noise[index],
                database.azi_noise[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate2 = f.find_orientation(measurements=measures2, params=(Ta, Tb, Tc))
        noise[index] = f.calc_disorientation(orientation, estimate2.x)

    # add columns
    dataset[columns[i]] = no_noise
    dataset[columns[i] + "_noise"] = noise

    # clean database
    del database.T_values
    del database.azimuths
    del database.T_noise
    del database.azi_noise


In [10]:
# check
# print('Euler angles: ', database.euler[-1])
# print('T values: ', np.around(database.T_values[-1], 2))
# print('Azimuths: ', np.around(database.azimuths[-1], 1))
# print('T noise: ', np.around(database.T_noise[-1], 2))
# print('Azi_noise: ', np.around(database.azi_noise[-1], 1))

In [11]:
dataset

,Euler1,Euler2,Euler3,n28,n28_noise,n32,n32_noise
0,0,0,0,0.243,10.735,0.106,0.093
1,0,0,7,0.000,10.292,0.249,13.677
2,0,0,14,0.000,7.822,0.000,0.030
3,0,0,21,0.303,14.547,0.232,0.952
4,0,0,28,0.000,4.318,0.277,0.891
...,...,...,...,...,...,...,...
4389,84,84,147,0.006,8.534,0.012,8.855
4390,84,84,154,0.009,8.496,0.007,6.021
4391,84,84,161,0.010,8.618,0.008,9.069
4392,84,84,168,0.011,5.406,0.013,6.279


## Estimate

In [12]:
from datetime import date    
today = date.today().isoformat()

In [13]:
dataset.to_csv(today + '_step7_lamb1987_noise_datapoints3.csv', index=False)

In [14]:
# quick statistical overview
dataset[['n28', 'n28_noise', 'n32', 'n32_noise']].describe()

,n28,n28_noise,n32,n32_noise
count,4394.000000,4394.000000,4394.000000,4394.000000
mean,0.027747,4.343968,0.030378,4.136903
std,0.093501,3.211963,0.138751,3.000243
min,0.000000,0.015000,0.000000,0.007000
25%,0.002000,2.023250,0.002000,1.912250
50%,0.004000,3.432500,0.004000,3.242000
75%,0.009000,5.938500,0.009000,5.745000
max,2.658000,41.378000,5.490000,17.915000
